# ANALISI E REVISIONE DEL PROGETTO LARSPLITTING 2D 

## CLASSE REFACTORING: boundingbox

Variabili utili per testare il funzionamento


In [13]:
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using IntervalTrees
using SparseArrays
using NearestNeighbors
using BenchmarkTools
using OrderedCollections
using Base.Threads


In [14]:
V = hcat([[0.,0],[1,0],[1,1],[0,1],[2,1]]...);    #vertici del modello 2D
V3 = hcat([[0.,0,0],[1,0,3],[1,1,2],[0,1,1],[2,1,0]]...);   #vertici del modello 3D
EV = [[1,2],[2,3],[3,4],[4,1],[1,5]];             #spigoli del modello
bb = [[0.0 1.0; 0.0 0.0], [1.0 1.0; 0.0 1.0], [0.0 1.0; 1.0 1.0], [0.0 0.0; 0.0 1.0], [0.0 2.0; 0.0 1.0]];  #bounding box
dict = OrderedDict([0.0, 1.0] => [1, 3],[1.0, 1.0] => [2],[0.0, 0.0] => [4],[0.0, 2.0] => [5])  #dizionario intervallo/indice
cov = [[4, 1, 3, 5, 2], [1, 3, 5, 2], [4, 1, 3, 5, 2], [4, 1, 3, 5], [4, 1, 3, 5, 2]]    #intersezioni tra bounding box

5-element Vector{Vector{Int64}}:
 [4, 1, 3, 5, 2]
 [1, 3, 5, 2]
 [4, 1, 3, 5, 2]
 [4, 1, 3, 5]
 [4, 1, 3, 5, 2]

## Versione iniziale di boundingbox

La funzione boundingbox serve a creare il bounding Box di una cella, cioè la scatola di misura più piccola (area, volume, ipervolume) entro cui sono contenuti tutti i punti.

In [15]:
function boundingbox(vertices::Lar.Points)
   minimum = mapslices(x->min(x...), vertices, dims=2)
   maximum = mapslices(x->max(x...), vertices, dims=2)
   return minimum, maximum
end

boundingbox (generic function with 1 method)

In [25]:
@btime boundingbox(V)   #20,202 μs

  20.202 μs (63 allocations: 2.19 KiB)


([0.0; 0.0;;], [2.0; 1.0;;])

### versione modificata boundingbox

In [19]:
function boundingboxMOD(vertices::Lar.Points)
    firstDim = vertices[1,:]
    secondDim = vertices[2,:]
     if (size(vertices,1)==3)
        thirdDim = vertices[3,:]
         minimum = Threads.@spawn hcat([min(firstDim...), min(secondDim...), min(thirdDim...)])
         maximum = Threads.@spawn hcat([max(firstDim...), max(secondDim...), max(thirdDim...)])
    else
         minimum = Threads.@spawn hcat([min(firstDim...), min(secondDim...)])
         maximum = Threads.@spawn hcat([max(firstDim...), max(secondDim...)])
    end
    return fetch(minimum),fetch(maximum)
 end

boundingboxMOD (generic function with 1 method)

In [24]:
@btime boundingboxMOD(V)   #13.282 μs

  13.282 μs (145 allocations: 5.00 KiB)


([0.0; 0.0;;], [2.0; 1.0;;])

### Benchmark della funzione iniziale e modificata

funzione iniziale:

In [22]:
@benchmark boundingbox(V)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  20.125 μs …  2.660 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     21.367 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   28.356 μs ± 52.409 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▆▂  ▁▂▂▂▂▃▃▁▁                                              ▁
  ████▇████████████▇▇█▆▆▆▆▅▆▆▅▅▆▅▅▅▄▃▄▄▅▅▅▄▄▄▃▄▄▃▄▃▂▅▂▄▃▃▃▃▃▃ █
  20.1 μs      Histogram: log(frequency) by time       102 μs <

 Memory estimate: 2.19 KiB, allocs estimate: 63.

funzione modificata:

In [23]:
@benchmark boundingboxMOD(V)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  13.162 μs …  8.770 ms  ┊ GC (min … max): 0.00% … 98.97%
 Time  (median):     14.031 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   18.212 μs ± 89.587 μs  ┊ GC (mean ± σ):  4.77% ±  0.99%

  █▇▃▁           ▁▁▁▁▁                                        ▁
  █████▇▆▆▆▆▆▆▆▇▇████████▇▆▆▅▆▅▅▅▅▄▄▄▄▄▄▄▅▄▅▅▂▃▃▄▅▃▄▃▃▃▄▃▃▄▄▄ █
  13.2 μs      Histogram: log(frequency) by time      68.7 μs <

 Memory estimate: 5.00 KiB, allocs estimate: 145.